In [26]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

In [27]:
start = dt.datetime.now()

# Use Keras Pretrained Models dataset

Kernels can't use network connection to download pretrained keras model weights.
This dataset helps you to apply your favorite pretrained model in the Kaggle Kernel environment. 
You can find more details [here](https://www.kaggle.com/gaborfodor/keras-pretrained-models).

We have to copy the pretrained models to the cache directory (~/.keras/models) where keras is looking for them.


In [28]:
!ls /home/chenxi-zhang/Downloads/keras-pretrained-models/

imagenet_class_index.json
inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5
inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5
inception_v3_weights_tf_dim_ordering_tf_kernels.h5
inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Kuszma.JPG
resnet50_weights_tf_dim_ordering_tf_kernels.h5
resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
xception_weights_tf_dim_ordering_tf_kernels.h5
xception_weights_tf_dim_ordering_tf_kernels_notop.h5


In [29]:
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [30]:
!cp /home/chenxi-zhang/Downloads/keras-pretrained-models/*notop* /home/chenxi-zhang/.keras/models/
!cp /home/chenxi-zhang/Downloads/keras-pretrained-models/imagenet_class_index.json /home/chenxi-zhang/.keras/models/
!cp /home/chenxi-zhang/Downloads/keras-pretrained-models/resnet50* /home/chenxi-zhang/.keras/models/

In [31]:
!ls /home/chenxi-zhang/.keras/models/

imagenet_class_index.json
inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5
inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
resnet50_weights_tf_dim_ordering_tf_kernels.h5
resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
xception_weights_tf_dim_ordering_tf_kernels_notop.h5


In [32]:
!ls /home/chenxi-zhang/Downloads/Dogs_Breed/

labels.csv  sample_submission.csv  test  test.zip  train  train.zip


# Use top 16 classes
Using all the images would take more than the 1 hour kernel limit. Let's focus on the most frequent 16 breeds.

In [35]:
INPUT_SIZE = 224
NUM_CLASSES = 16
SEED = 1987
data_dir = "/home/chenxi-zhang/Downloads/Dogs_Breed/"
labels = pd.read_csv(join(data_dir, 'labels.csv'))
sample_submission = pd.read_csv(join(data_dir, 'sample_submission.csv'))
print(len(listdir(join(data_dir, 'train'))), len(labels))
print(len(listdir(join(data_dir, 'test'))), len(sample_submission))

10222 10222
10357 10357


In [46]:
selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)
labels = labels[labels['breed'].isin(selected_breed_list)]
labels['target'] = 1
labels['rank'] = labels.groupby('breed').rank()['id']
#labels_pivot = labels.pivot('id', 'breed', 'target').reset_index().fillna(0)
#np.random.seed(seed=SEED)
#rnd = np.random.random(len(labels))
#train_idx = rnd < 0.8
#valid_idx = rnd >= 0.8
#y_train = labels_pivot[selected_breed_list].values
#ytr = y_train[train_idx]
#yv = y_train[valid_idx]

TypeError: 'NoneType' object is not callable